In [2]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets



In [34]:
# Run this code if you get errors with missing requirements
import sys
!{sys.executable} -m pip install mysql-connector

import pandas as pd
import mysql.connector

mydb = mysql.connector.connect(
  host="iiwari-mariadb-server",
  user="root",
  passwd="d41k4Duu",
  database="iiwari_org"
)

In [35]:
mycursor = mydb.cursor(dictionary=True)
mycursor.execute("SELECT node_id, timestamp, x, y FROM SensorData WHERE timestamp > '2019-05-25 04:00:08.078133+00:00' AND timestamp < '2019-06-01 20:00:08.078133+00:00'  AND x > 1000  AND X < 10406 AND y < 5220 AND y > 0  AND z = 0 LIMIT 10000");
database = pd.DataFrame(mycursor.fetchall())
print(database)

      node_id                            timestamp     x     y
0       52535   b'2019-05-25 05:14:15.98762+00:00'  1027  2872
1       52535  b'2019-05-25 05:14:16.987661+00:00'  1132  2858
2       52535  b'2019-05-25 05:14:17.988363+00:00'  1249  2864
3       52535  b'2019-05-25 05:14:18.987879+00:00'  1348  2864
4       52535   b'2019-05-25 05:14:19.98964+00:00'  1591  2847
...       ...                                  ...   ...   ...
9995    51735   b'2019-05-25 07:58:24.99257+00:00'  6594  2036
9996    51735  b'2019-05-25 07:58:25.987437+00:00'  7437  1780
9997    52003  b'2019-05-25 07:58:25.987444+00:00'  9410  1173
9998    52003  b'2019-05-25 07:58:25.987447+00:00'  9410  1173
9999    52535  b'2019-05-25 07:58:25.988533+00:00'  2736   145

[10000 rows x 4 columns]


In [36]:

nodes = pd.DataFrame()
nodes['node_id'] = database['node_id']
database['timestamp'] = database['timestamp'].astype('str')
database['timestamp'] = database['timestamp'].str.rstrip("'+00:00'")
database['timestamp'] = database['timestamp'].str.lstrip("b'")
database['timestamp'] = database['timestamp'].astype('datetime64[s]')

database.timestamp = database.timestamp.dt.tz_localize('UTC')
database.timestamp = database.timestamp.dt.tz_convert('Europe/Helsinki')

nodes['timestamp'] = database['timestamp']

database = database.drop(database[(database.timestamp.dt.hour > 21)].index)
database = database.drop(database[(database.timestamp.dt.hour < 8)].index)


nodes.index = nodes['timestamp']
nodes = nodes.drop('timestamp', axis=1)

nodes = nodes.resample('H').count()

nodes['esintyvyysprosentti'] = 100 * (nodes['node_id'] / nodes['node_id'].sum())

nodes.index = nodes.index.astype(str)
nodes.index = nodes.index.str.slice(10, -6)

print(nodes)

           node_id  esintyvyysprosentti
timestamp                              
 08:00:00      410                 4.10
 09:00:00     1843                18.43
 10:00:00     7747                77.47


In [37]:
database['kuukaudet'] = database.timestamp.dt.month
print(database)

      node_id                 timestamp     x     y  kuukaudet
0       52535 2019-05-25 08:14:15+03:00  1027  2872          5
1       52535 2019-05-25 08:14:16+03:00  1132  2858          5
2       52535 2019-05-25 08:14:17+03:00  1249  2864          5
3       52535 2019-05-25 08:14:18+03:00  1348  2864          5
4       52535 2019-05-25 08:14:19+03:00  1591  2847          5
...       ...                       ...   ...   ...        ...
9995    51735 2019-05-25 10:58:24+03:00  6594  2036          5
9996    51735 2019-05-25 10:58:25+03:00  7437  1780          5
9997    52003 2019-05-25 10:58:25+03:00  9410  1173          5
9998    52003 2019-05-25 10:58:25+03:00  9410  1173          5
9999    52535 2019-05-25 10:58:25+03:00  2736   145          5

[10000 rows x 5 columns]


In [45]:
interact(f, aika=(8,21))

interactive(children=(IntSlider(value=14, description='aika', max=21, min=8), Output()), _dom_classes=('widget…

<function __main__.f(aika)>